# Instructions

The places where you have enter code, to answer the questions, are marked with `# YOUR CODE HERE`. Once you have written your code you should remove the `raise NotImplementedError()` statement.

## Question 1 (1 point)

Complete the function `calcL` such that given $N$, it returns the size $L$ of the target register. Note that $L = \log_2 N$. 

The function `calcL` has

- Input: integer $N$
- Returns: integer L, the size of the target register

In [35]:
import math
def calcL(N):
    
    # YOUR CODE HERE
    L=math.ceil(math.log2(N))
    #raise NotImplementedError()

    return L

In [36]:
# You can use this cell to test your function
print(calcL(30))
print(calcL(62))

5
6


In [ ]:
# hidden tests in this cell will be used for grading.

## Question 2 (1 point)

Complete the function `calct` such that given $N$, it returns the size $t$ of the control register. Note that $t = 2L + 1 + \big \lceil \log \big( 2 + \frac{1}{2\epsilon} \big) \big \rceil$. Let $\epsilon=0.1$. 

The function `calct` has

- Input: integer $N$
- Returns: integer t, size of the control register

In [29]:
import math
def calct(N):
    
    # YOUR CODE HERE
    eps = 0.1
    L=math.ceil(math.log2(N))
    t=2*L+1+math.ceil(math.log2(1+1/(2*eps)))
    #raise NotImplementedError()
    
    return t

In [30]:
# You can use this cell to test your function
print(calct(30)) #For 30 the answer is 14
print(calct(62)) #For 62 the answer is 16

14
16


In [ ]:
# hidden tests in this cell will be used for grading.

## Question 3 (2 points)

Complete the function `create_regs` such that given the size of the control and target registers, it returns the control and target registers.

The function `create_regs` has

- Input: integers $L$ and $t$
- Returns: Two registers `control` and `target`


In [ ]:
import cirq
def create_regs(t,L):

    # YOUR CODE HERE
    
    #raise NotImplementedError()
    
    return control, target

In [ ]:
# hidden tests in this cell will be used for grading.

In [ ]:
# hidden tests in this cell will be used for grading.

## Question 4 (6 points)

Complete the function `order` so that given $x$ and $N$, it returns the histogram of samples from the order finding circuit.

The function `order` has

- Input: $x$ and $N$
- Returns: A histogram of samples

Notes:

- You are given a function named $U_x$ which implements $U_x |y\rangle \rightarrow |xy {\mod N}\rangle $ and returns its controlled version. Run the following cell to load the function. In order to use the function you should pass $x$ and $N$ as parameters.

    <pre>CU=Ux(x,N)</pre>

- You are also given the code for `qpe` and `iqft`.

In [31]:
import numpy as np
def Ux(x,N):

    k=1
    while(N>2**k):
        k=k+1
        
    u = np.zeros([2**k, 2**k], dtype = int) 

    for i in range(N):
        u[x*i%N][i]=1
    for i in range(N,2**k):
        u[i][i]=1
        

    XU = cirq.MatrixGate(u).controlled()
    return XU


In [32]:
import cirq
from cirq.circuits import InsertStrategy
from cirq import H, SWAP, CZPowGate

def iqft(n,qubits,circuit):
 
    #Swap the qubits
    for i in range(n//2):
        circuit.append(SWAP(qubits[i],qubits[n-i-1]), strategy = InsertStrategy.NEW)
     
    #For each qubit
    for i in range(n-1,-1,-1):
        #Apply CR_k gates where j is the control and i is the target
        k=n-i #We start with k=n-i
        for j in range(n-1,i,-1):
            #Define and apply CR_k gate
            crk = CZPowGate(exponent = -2/2**(k))
            circuit.append(crk(qubits[j],qubits[i]),strategy = InsertStrategy.NEW)
            k=k-1 #Decrement at each step
            
        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]),strategy = InsertStrategy.NEW)

def qpe(t,control, target, circuit, CU):
    
    #Apply Hadamard to control qubits
    circuit.append(cirq.H.on_each(control))
    
    #Apply CU gates
    for i in range(t):
        #Obtain the power of CU gate 
        CUi = CU**(2**i)
        #Apply CUi gate where t-i-1 is the control
        circuit.append(CUi(control[t-i-1],*target))
        
    #Apply inverse QFT
    iqft(t,control,circuit)

In [33]:
import cirq
from cirq import X, SWAP

def order(x,N):
    #Create a circuit
    circuit = cirq.Circuit()
    
    # YOUR CODE HERE
    L=math.ceil(math.log2(N))
    e = 0.1
    t = 2*L+1+math.ceil(math.log2(2+1/(2*e)))
    control = [cirq.LineQubit(i) for i in range(1,t+1) ]
    target = [cirq.LineQubit(i) for i in range(t+1,t+1+L) ]
    
    circuit.append(cirq.X(target[L-1]))

    #Create operator CU
    CU=Ux(x,N)

    #Call phase estimation circuit
    qpe(t,control, target, circuit, CU)    
    
    #raise NotImplementedError()

    #Measure the control register
    circuit.append(cirq.measure(*control, key='result'))

    #Sample the circuit
    s=cirq.Simulator()
    samples=s.run(circuit, repetitions=1000)

    # Return a histogram of samples
    results= samples.histogram(key='result')
    return results

In [34]:
#You can use this cell to test your function

results = order(4,31)
print(results)

Counter({3277: 205, 0: 187, 13107: 171, 9830: 130, 6554: 94, 6553: 53, 9831: 51, 13108: 13, 6555: 10, 6552: 9, 3276: 9, 3278: 6, 13106: 6, 9832: 5, 9833: 4, 9829: 4, 13110: 3, 6530: 2, 6551: 2, 6520: 1, 9834: 1, 6559: 1, 13210: 1, 3290: 1, 9826: 1, 13113: 1, 9797: 1, 9837: 1, 3275: 1, 9827: 1, 6613: 1, 13111: 1, 9838: 1, 6556: 1, 6550: 1, 9828: 1, 3263: 1, 9815: 1, 13112: 1, 3273: 1, 9825: 1, 13093: 1, 3285: 1, 6557: 1, 9794: 1, 6575: 1, 6529: 1, 6590: 1, 9758: 1, 6495: 1, 3279: 1, 9851: 1, 6545: 1, 6528: 1, 13109: 1})


In [ ]:
# hidden tests in this cell will be used for grading.